# Урок 3. Системы управления базами данных MongoDB и SQLite в Python

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД (без датафрейма)

In [1]:
from pymongo import MongoClient
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import re
import datetime
from lxml import etree

In [2]:
client = MongoClient('localhost',27017)
db = client['vacancies_db']
vacancies = db.vacancies

In [3]:
def parser_list_hh(soup):
    vacancy_block = soup.find('div',{'class':'vacancy-serp'})
    vacancy_list = vacancy_block.find_all('div',{'class':'vacancy-serp-item'})
    vacancies = []
    for vacancy in vacancy_list:
        vacancy_data = {}
#         vacancy_params = eval(re.sub(r'^\s+|\n|\r|\s+$', '', vacancy.find('script', 
#                                                                             {'data-name':'HH/Vacancy/SendResponseAttempt'})
#                                                                             ['data-params'] \
#                                                                             .replace(' ', '')))
  
#         vacancy_id = int(vacancy_params['vacancyId'])
        vacancy_name = vacancy.find('a', {'class':'bloko-link'}).getText()
        vacancy_link = vacancy.find('a',{'class':'bloko-link'})['href']
        vacancy_salary = vacancy.find('div', {'class':'vacancy-serp-item__sidebar'}).getText()
        if not vacancy_salary:
            vacancy_salary_min = None
            vacancy_salary_max = None
            vacancy_salary_currency = None
        else:
            vacancy_salary = vacancy_salary \
                            .replace(u'\xa0', u'')

            vacancy_salary = re.split(r'\s|-', vacancy_salary)

            if vacancy_salary[0] == 'до':
                vacancy_salary_min = None
                vacancy_salary_max = int(vacancy_salary[1])
            elif vacancy_salary[0] == 'от':
                vacancy_salary_min = int(vacancy_salary[1])
                vacancy_salary_max = None
            else:
                vacancy_salary_min = int(vacancy_salary[0])
                vacancy_salary_max = int(vacancy_salary[1])            

            vacancy_salary_currency = vacancy_salary[2]
        
#         vacancy_data['_id'] = vacancy_id
        vacancy_data['name'] = vacancy_name
        vacancy_data['link'] = vacancy_link
        vacancy_data['salary_min'] = vacancy_salary_min
        vacancy_data['salary_max'] = vacancy_salary_max
        vacancy_data['salary_currency'] = vacancy_salary_currency
        vacancy_data['site'] = 'hh.ru'

        vacancies.append(vacancy_data)
    return vacancies

In [4]:
def parser_hh(query):
    hh_main_link = 'https://hh.ru'
    query = query
    hh_page = 0
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'}

    hh_vacancy_date = []

    while True:
        hh_response = requests.get(hh_main_link + '/search/vacancy?text=' + query + '&page=' + str(hh_page), headers=headers).text
        hh_soup = bs(hh_response,'lxml')
        vacancies.insert_many(parser_list_hh(hh_soup))
        hh_page += 1
        if hh_soup.find('a',{'class':'HH-Pager-Controls-Next'}) is None:
            break

In [5]:
query = 'Python'
parser_hh(query)

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

In [21]:
def print_salary(salary):
    currency = {'USD': 'R01235',
            'EUR': 'R01239',
            'KZT': 'R01335',
            'бел.руб.': 'R01090B',
            'сум': 'R01717',
            'грн.': 'R01720'}

    now_time = datetime.datetime.now().strftime("%d/%m/%Y")
    url = 'http://www.cbr.ru/scripts/XML_dynamic.asp'


    currency_k = {'руб.': 1, None: 1}
    for key in currency:
        params = {'date_req1': now_time, 'date_req2': now_time, 'VAL_NM_RQ': currency[key]}
        k = requests.get(url, params=params).text.encode('utf8')
        root = etree.fromstring(k)
        for appt in root.getchildren():
            for elem in appt.getchildren():
                if elem.tag == 'Value':
                    value = elem.text
                else:
                    nominal = elem.text
            currency_k[key] = float(value.replace(',', '.'))/int(nominal)
    
    print(currency_k)
    for vacancy in vacancies.find \
        ({'$or' : 
        [{'salary_currency': 'руб.', '$or' : [{'salary_min': {'$gt' : salary}}, {'salary_max': {'$gt' : salary}}]},
        {'salary_currency': 'USD', '$or' : [{'salary_min': {'$gt' : salary / currency_k['USD']}}, {'salary_max': {'$gt' : salary / currency_k['USD']}}]},
        {'salary_currency': 'EUR', '$or' : [{'salary_min': {'$gt' : salary / currency_k['EUR']}}, {'salary_max': {'$gt' : salary / currency_k['EUR']}}]},
        {'salary_currency': 'KZT', '$or' : [{'salary_min': {'$gt' : salary / currency_k['KZT']}}, {'salary_max': {'$gt' : salary / currency_k['KZT']}}]},
        {'salary_currency': 'бел.руб.', '$or' : [{'salary_min': {'$gt' : salary / currency_k['бел.руб.']}}, {'salary_max': {'$gt' : salary / currency_k['бел.руб.']}}]},
        {'salary_currency': 'сум', '$or' : [{'salary_min': {'$gt' : salary / currency_k['сум']}}, {'salary_max': {'$gt' : salary / currency_k['сум']}}]},
        {'salary_currency': 'грн.', '$or' : [{'salary_min': {'$gt' : salary / currency_k['грн.']}}, {'salary_max': {'$gt' : salary / currency_k['грн.']}}]}]}):
        print(vacancy)

In [22]:
salary = 300000
print_salary(salary)

{'руб.': 1, None: 1, 'USD': 70.4999, 'EUR': 79.2207, 'KZT': 0.173344, 'бел.руб.': 29.0183, 'сум': 0.0069298400000000005, 'грн.': 2.59262}
{'_id': ObjectId('5efe109955ac8396c2d6165c'), 'name': 'Senior Machine Learning Engineer', 'link': 'https://khimki.hh.ru/vacancy/37703126?query=Python', 'salary_min': 250000, 'salary_max': 450000, 'salary_currency': 'руб.', 'site': 'hh.ru'}
{'_id': ObjectId('5efe109955ac8396c2d61668'), 'name': 'Backend-разработчик', 'link': 'https://khimki.hh.ru/vacancy/37535012?query=Python', 'salary_min': 180000, 'salary_max': 350000, 'salary_currency': 'руб.', 'site': 'hh.ru'}
{'_id': ObjectId('5efe109a55ac8396c2d616ce'), 'name': 'Senior DevOps Engineer (Belgrade, Serbia)', 'link': 'https://khimki.hh.ru/vacancy/37400339?query=Python', 'salary_min': 5000, 'salary_max': 7000, 'salary_currency': 'USD', 'site': 'hh.ru'}
{'_id': ObjectId('5efe109b55ac8396c2d616f0'), 'name': 'Representation Learning Senior Research Scientist', 'link': 'https://khimki.hh.ru/vacancy/371438

3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [10]:
def parser_list_hh_new(soup):
    vacancy_block = soup.find('div',{'class':'vacancy-serp'})
    vacancy_list = vacancy_block.find_all('div',{'class':'vacancy-serp-item'})
    for vacancy in vacancy_list:
        vacancy_data = {}
        vacancy_params = eval(re.sub(r'^\s+|\n|\r|\s+$', '', vacancy.find('script', 
                                                                            {'data-name':'HH/Vacancy/SendResponseAttempt'})
                                                                            ['data-params'] \
                                                                            .replace(' ', '')))
  
        vacancy_id = int(vacancy_params['vacancyId'])
        vacancy_name = vacancy.find('a', {'class':'bloko-link'}).getText()
        vacancy_link = vacancy.find('a',{'class':'bloko-link'})['href']
        vacancy_salary = vacancy.find('div', {'class':'vacancy-serp-item__sidebar'}).getText()
        if not vacancy_salary:
            vacancy_salary_min = None
            vacancy_salary_max = None
            vacancy_salary_currency = None
        else:
            vacancy_salary = vacancy_salary \
                            .replace(u'\xa0', u'')

            vacancy_salary = re.split(r'\s|-', vacancy_salary)

            if vacancy_salary[0] == 'до':
                vacancy_salary_min = None
                vacancy_salary_max = int(vacancy_salary[1])
            elif vacancy_salary[0] == 'от':
                vacancy_salary_min = int(vacancy_salary[1])
                vacancy_salary_max = None
            else:
                vacancy_salary_min = int(vacancy_salary[0])
                vacancy_salary_max = int(vacancy_salary[1])            

            vacancy_salary_currency = vacancy_salary[2]
        
        vacancy_data['_id'] = vacancy_id
        vacancy_data['name'] = vacancy_name
        vacancy_data['link'] = vacancy_link
        vacancy_data['salary_min'] = vacancy_salary_min
        vacancy_data['salary_max'] = vacancy_salary_max
        vacancy_data['salary_currency'] = vacancy_salary_currency
        vacancy_data['site'] = 'hh.ru'
        
        for vacancy in vacancies.find({'link': vacancy_data['link']}):
            if True: 
                collection.replace_one({'link' : vacancy_data['link']}, vacancy_data)
        vacancies.replace_one({'_id' : vacancy_data['_id']}, vacancy_data, True)  
    return vacancies

In [11]:
def parser_hh_new(query):
    hh_main_link = 'https://hh.ru'
    query = query
    hh_page = 0
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'}

    hh_vacancy_date = []

    while True:
        hh_response = requests.get(hh_main_link + '/search/vacancy?text=' + query + '&page=' + str(hh_page), headers=headers).text
        hh_soup = bs(hh_response,'lxml')
        parser_list_hh(hh_soup)
        hh_page += 1
        if hh_soup.find('a',{'class':'HH-Pager-Controls-Next'}) is None:
            break

In [12]:
query = 'Python'
parser_hh_new(query)

In [14]:
for i in vacancies.find({}):
    pprint(i)


{'_id': ObjectId('5efe109955ac8396c2d61640'),
 'link': 'https://khimki.hh.ru/vacancy/37765447?query=Python',
 'name': 'Скилловый Python Developer на бэкенд',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 250000,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109955ac8396c2d61641'),
 'link': 'https://khimki.hh.ru/vacancy/37461514?query=Python',
 'name': 'Python / Django fullstack программист',
 'salary_currency': 'руб.',
 'salary_max': 180000,
 'salary_min': 110000,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109955ac8396c2d61642'),
 'link': 'https://khimki.hh.ru/vacancy/37552725?query=Python',
 'name': 'Middle software developer (ML)',
 'salary_currency': 'KZT',
 'salary_max': 500000,
 'salary_min': 300000,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109955ac8396c2d61643'),
 'link': 'https://khimki.hh.ru/vacancy/37792411?query=Python',
 'name': 'Python Developer',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109955ac8396

 'salary_max': 200000,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109c55ac8396c2d61723'),
 'link': 'https://khimki.hh.ru/vacancy/37394076?query=Python',
 'name': 'Разработчик баз данных (MS SQL)',
 'salary_currency': 'руб.',
 'salary_max': 150000,
 'salary_min': 120000,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109c55ac8396c2d61724'),
 'link': 'https://khimki.hh.ru/vacancy/37313177?query=Python',
 'name': 'Эксперт по Информационной безопасности (Пентестер)',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 100000,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109c55ac8396c2d61725'),
 'link': 'https://khimki.hh.ru/vacancy/37458552?query=Python',
 'name': 'Разработчик (интеграция ПО)',
 'salary_currency': 'руб.',
 'salary_max': 300000,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109c55ac8396c2d61726'),
 'link': 'https://khimki.hh.ru/vacancy/37379802?query=Python',
 'name': 'Стажер (Голосовая Биометрия)',
 'salary_currency': 'руб.',
 'salary_max':

 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109f55ac8396c2d6180c'),
 'link': 'https://khimki.hh.ru/vacancy/37394042?query=Python',
 'name': 'Python Developer',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109f55ac8396c2d6180d'),
 'link': 'https://khimki.hh.ru/vacancy/37331855?query=Python',
 'name': 'Бизнес-аналитик (web and mobile)',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 160000,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109f55ac8396c2d6180e'),
 'link': 'https://khimki.hh.ru/vacancy/35594122?query=Python',
 'name': 'Technical Support Engineer (Nx Meta)',
 'salary_currency': 'USD',
 'salary_max': None,
 'salary_min': 3000,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe109f55ac8396c2d6180f'),
 'link': 'https://khimki.hh.ru/vacancy/37117154?query=Python',
 'name': 'Преподаватель программирования для детей и подростков (Алгоритмика)',
 'salary_currency': 'KZT',
 'salary_max': 400000,
 'salary_

 'link': 'https://khimki.hh.ru/vacancy/37619840?query=Python',
 'name': 'Middle / Senior Python Developer',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe10a355ac8396c2d618fd'),
 'link': 'https://khimki.hh.ru/vacancy/37127951?query=Python',
 'name': 'Программист С++, Python',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe10a355ac8396c2d618fe'),
 'link': 'https://khimki.hh.ru/vacancy/32876723?query=Python',
 'name': 'Senior Python Developer',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe10a355ac8396c2d618ff'),
 'link': 'https://khimki.hh.ru/vacancy/37558882?query=Python',
 'name': 'Программист-разработчик Python',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe10a355ac8396c2d61900'),
 'link': 'https://khimki.hh.ru/vacancy/37499273?query=Python

{'_id': ObjectId('5efe10a655ac8396c2d619ff'),
 'link': 'https://khimki.hh.ru/vacancy/35268660?query=Python',
 'name': 'Системный администратор Unix',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe10a655ac8396c2d61a00'),
 'link': 'https://khimki.hh.ru/vacancy/37797908?query=Python',
 'name': 'Data-инженер «Инструменты управления рисками и потерями в Сети '
         'продаж»',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe10a655ac8396c2d61a01'),
 'link': 'https://khimki.hh.ru/vacancy/36211911?query=Python',
 'name': 'Программист C++/Qt',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 62500,
 'site': 'hh.ru'}
{'_id': ObjectId('5efe10a655ac8396c2d61a02'),
 'link': 'https://khimki.hh.ru/vacancy/37547178?query=Python',
 'name': 'Python Backend Developer',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': Objec

...

 'site': 'hh.ru'}
{'_id': ObjectId('5f02236ef7e4d0d8abe30e38'),
 'link': 'https://khimki.hh.ru/vacancy/36886171?query=Python',
 'name': 'Senior Python Engineer',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f02236ef7e4d0d8abe30e39'),
 'link': 'https://khimki.hh.ru/vacancy/35366975?query=Python',
 'name': 'Программист Python',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f02236ef7e4d0d8abe30e3a'),
 'link': 'https://khimki.hh.ru/vacancy/37572391?query=Python',
 'name': 'Старший Fullstack разработчик (Python, Django, React)',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f02236ef7e4d0d8abe30e3b'),
 'link': 'https://khimki.hh.ru/vacancy/36550269?query=Python',
 'name': 'Python Developer (AI)',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f02236ef7e4d0d8ab

 'link': 'https://khimki.hh.ru/vacancy/37681863?query=Python',
 'name': 'Backend Developer',
 'salary_currency': 'руб.',
 'salary_max': 80000,
 'salary_min': 45000,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022371f7e4d0d8abe30f39'),
 'link': 'https://khimki.hh.ru/vacancy/37579629?query=Python',
 'name': 'DevOps инженер',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 220000,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022371f7e4d0d8abe30f3a'),
 'link': 'https://khimki.hh.ru/vacancy/37816959?query=Python',
 'name': 'Разработчик драйверов Linux (Junior)',
 'salary_currency': 'руб.',
 'salary_max': 90000,
 'salary_min': 60000,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022371f7e4d0d8abe30f3b'),
 'link': 'https://khimki.hh.ru/vacancy/37334033?query=Python',
 'name': 'Product Owner',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022371f7e4d0d8abe30f3c'),
 'link': 'https://khimki.hh.ru/vacancy/37771863?query=Python',
 'name': 'Гла

 'link': 'https://khimki.hh.ru/vacancy/37816707?query=Python',
 'name': 'Middle+/Senior PHP Backend Developer',
 'salary_currency': 'руб.',
 'salary_max': 180000,
 'salary_min': 100000,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022376f7e4d0d8abe31043'),
 'link': 'https://khimki.hh.ru/vacancy/37425106?query=Python',
 'name': 'Программист С++',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022376f7e4d0d8abe31044'),
 'link': 'https://khimki.hh.ru/vacancy/37327073?query=Python',
 'name': 'Senior Python developer',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022376f7e4d0d8abe31045'),
 'link': 'https://khimki.hh.ru/vacancy/36497952?query=Python',
 'name': 'Tech lead Backend',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022376f7e4d0d8abe31046'),
 'link': 'https://khimki.hh.ru/vacancy/37781823?query=Python',
 'name':

 'name': 'Системный администратор / DevOps (удалённо)',
 'salary_currency': 'руб.',
 'salary_max': 70000,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022379f7e4d0d8abe3114d'),
 'link': 'https://khimki.hh.ru/vacancy/36411748?query=Python',
 'name': 'Ведущий системный инженер Machine Learning платформы / DevOps',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022379f7e4d0d8abe3114e'),
 'link': 'https://khimki.hh.ru/vacancy/37814358?query=Python',
 'name': 'Ведущий разработчик RPA (Python)',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022379f7e4d0d8abe3114f'),
 'link': 'https://khimki.hh.ru/vacancy/37813003?query=Python',
 'name': 'Senior Deep Learning Software developer/Architect',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f022379f7e4d0d8abe31150'),
 'link': 'https://khimki.hh.ru/vacancy/3604

 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f02237ef7e4d0d8abe31257'),
 'link': 'https://khimki.hh.ru/vacancy/37762427?query=Python',
 'name': 'Python разработчик (удаленная работа)',
 'salary_currency': 'руб.',
 'salary_max': 200000,
 'salary_min': 120000,
 'site': 'hh.ru'}
{'_id': ObjectId('5f02237ef7e4d0d8abe31258'),
 'link': 'https://khimki.hh.ru/vacancy/37667286?query=Python',
 'name': 'Специалист по адаптации игрового контента',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f02237ef7e4d0d8abe31259'),
 'link': 'https://khimki.hh.ru/vacancy/37770403?query=Python',
 'name': 'NLP Software Developer / NLP программист-разработчик',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}
{'_id': ObjectId('5f02237ef7e4d0d8abe3125a'),
 'link': 'https://khimki.hh.ru/vacancy/34012100?query=Python',
 'name': 'Программист Python',
 'salary_currency': None,
 'salary_max':